In [16]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

In [17]:
#reading vaclogger measurement file
vaclog1=pd.read_csv("vaclog during bakeout", sep="\t")
vaclog2=pd.read_csv("vaclog during bakeout_fil2", sep="\t")
vaclog=pd.concat([vaclog1,vaclog2], ignore_index=True)
vaclog.head()

#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,V_heater,I_emission,I_grid
0,NaN,2023-04-24 11:20:11,0.0,-0.053024,1.140000e-07,4.080000e-08,5.454000e-08,0.000005,-0.000828,0.0,108.080,57.014,-0.000044,NaN,NaN
1,NaN,2023-04-24 11:20:18,7.0,-0.053492,1.140000e-07,4.080000e-08,5.454000e-08,0.000005,-0.000836,0.0,108.193,57.014,-0.000044,NaN,NaN
2,NaN,2023-04-24 11:20:26,15.0,-0.053146,6.090000e-06,7.600000e-07,3.656000e-06,0.000005,-0.000833,0.0,108.305,57.014,-0.000044,NaN,NaN
3,NaN,2023-04-24 11:20:34,23.0,-0.053295,4.230000e-06,6.060000e-07,2.110000e-06,0.000005,-0.000834,0.0,108.305,57.014,-0.000044,NaN,NaN
4,NaN,2023-04-24 11:20:41,30.0,-0.053418,1.990000e-06,3.180000e-07,1.020000e-06,0.000005,-0.000830,0.0,108.417,57.127,-0.000044,NaN,NaN


In [18]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [19]:
#print hv comments
print(pd.unique(vaclog["Live comments"]))
vaclog_comm = vaclog.loc[:,["Time","Live comments"]]
vaclog_comm.dropna(inplace=True)
print(vaclog_comm)
vaclog_comm.index

[nan 'baryons off' 'starting bakeout' 'bakeout started'
 'agilent daq card reseated' 'degasin barions']
                     Time              Live comments
21    2023-04-24 11:22:52                baryons off
22    2023-04-24 11:23:00           starting bakeout
149   2023-04-24 11:39:30            bakeout started
1264  2023-04-24 14:09:22  agilent daq card reseated
12640 2023-04-25 14:19:39            degasin barions


Int64Index([21, 22, 149, 1264, 12640], dtype='int64')

### Plotting

In [20]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [21]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.plot(vaclog["Time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='Dual exp')

plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution during bakeout")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')  

#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in vaclog_comm.index:
    plt.text(vaclog_comm["Time"][i],0.8,str(vaclog_comm["Live comments"][i]),fontdict=font, transform=trans)
    
plt.savefig(r'./graphs/Baryons_bakeout.png')
plt.show()

In [22]:
## Plotting the temp evolution
plt.figure(figsize=(10,6))
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution during bakeout")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.savefig(r'./graphs/Temp_bakeout.png')
plt.show()